In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

In [4]:
path ='UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt'

In [6]:
# read_file(path)

In [7]:
import spacy

In [11]:
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

In [12]:
nlp.max_length = 1198623

In [18]:
def sep_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [19]:
d = read_file(path)

In [20]:
tokens = sep_punc(d)

In [21]:
len(tokens)

11394

In [22]:
# 25 words --> network predict 26

In [24]:
train_len = 25 + 1

text_seq = []

for i in range(train_len,len(tokens)):
    seq = tokens[i - train_len:i]
    text_seq.append(seq)


In [31]:
' '.join(text_seq[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [32]:
' '.join(text_seq[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [36]:
from keras.preprocessing.text import Tokenizer

In [38]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_seq)

In [39]:
seqs = tokenizer.texts_to_sequences(text_seq)

In [42]:
seqs[1]

[14,
 265,
 51,
 263,
 416,
 87,
 222,
 129,
 111,
 962,
 262,
 50,
 43,
 37,
 321,
 7,
 23,
 555,
 3,
 150,
 261,
 6,
 2704,
 14,
 24,
 965]

In [73]:
d = create_dict(tokenizer.word_index)
for i in seqs[0]:
    print(i,d[i])

964 call
14 me
265 ishmael
51 some
263 years
416 ago
87 never
222 mind
129 how
111 long
962 precisely
262 having
50 little
43 or
37 no
321 money
7 in
23 my
555 purse
3 and
150 nothing
261 particular
6 to
2704 interest
14 me
24 on


In [71]:
def create_dict(d):
    p = {}
    for k, v in d.items():
        p[v] = k
    return p

In [74]:
vocab_size = len(tokenizer.word_counts)

In [75]:
vocab_size

2709

In [76]:
import numpy as np

In [78]:
seqs = np.array(seqs)

In [80]:
seqs

array([[ 964,   14,  265, ..., 2704,   14,   24],
       [  14,  265,   51, ...,   14,   24,  965],
       [ 265,   51,  263, ...,   24,  965,    5],
       ...,
       [ 960,   12,  168, ...,  264,   53,    2],
       [  12,  168, 2703, ...,   53,    2, 2709],
       [ 168, 2703,    3, ...,    2, 2709,   26]])

In [81]:
from keras.utils import to_categorical 

In [84]:
X = seqs[:,:-1]

In [88]:
y = seqs[:,-1]

In [89]:
y = to_categorical(y,num_classes=vocab_size+1)

In [90]:
seq_len = X.shape[1]

In [91]:
X.shape

(11368, 25)

In [92]:
from keras.models import Sequential
from keras.layers import Dense, LSTM,Embedding

In [97]:
def create_model(vocab_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocab_size,seq_len,input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))
    
    model.add(Dense(vocab_size, activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [99]:
model = create_model(vocab_size+1,seq_len)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 25, 25)            67750     
_________________________________________________________________
lstm_4 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 2710)              138210    
Total params: 243,910
Trainable params: 243,910
Non-trainable params: 0
_________________________________________________________________


In [100]:
from pickle import dump,load

In [123]:
model.fit(X,y,batch_size=128,epochs=10,verbose=1)

Epoch 1/10
11368/11368 [==============================] - 12s 1ms/step - loss: 6.3450 - acc: 0.0527
Epoch 2/10
11368/11368 [==============================] - 11s 950us/step - loss: 6.3378 - acc: 0.0527
Epoch 3/10
11368/11368 [==============================] - 10s 916us/step - loss: 6.3305 - acc: 0.0527
Epoch 4/10
11368/11368 [==============================] - 11s 936us/step - loss: 6.3005 - acc: 0.0527
Epoch 5/10
11368/11368 [==============================] - 11s 930us/step - loss: 6.1822 - acc: 0.0527
Epoch 6/10
11368/11368 [==============================] - 11s 941us/step - loss: 6.1104 - acc: 0.0527
Epoch 7/10
11368/11368 [==============================] - 9s 821us/step - loss: 6.0744 - acc: 0.0527
Epoch 8/10
11368/11368 [==============================] - 9s 753us/step - loss: 6.0463 - acc: 0.0527
Epoch 9/10
11368/11368 [==============================] - 9s 821us/step - loss: 6.0193 - acc: 0.0538
Epoch 10/10
11368/11368 [==============================] - 9s 824us/step - loss: 5.9943

In [102]:
model.save('my_mobydick_model.h5')

In [103]:
dump(tokenizer,open('my_simpletokenizer','wb'))

In [104]:
from keras.preprocessing.sequence import pad_sequences

In [118]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    output_text = []
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_len,truncating='pre')
        pred_word_ind = model.predict_classes(pad_encoded,verbose=0)[0]
        pred_word = create_dict(tokenizer.word_index)[pred_word_ind]
        input_text += ' '+ pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)
        

In [119]:
import random
random.seed(101)
ran_pick = random.randint(0,len(text_seq))

In [120]:
seed_text = ' '.join(text_seq[ran_pick])

In [121]:
seed_text

'and throwing the clothes to one side he really did this in not only a civil but a really kind and charitable way i stood looking'

In [122]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'the the the the the the the the the the the the the the the the the the the the the the the the the'

In [4]:
def fact(n):
    if n <2:
        return 1
    return n* fact(n-1)

In [6]:
print(fact(29) / (fact(24) * fact(5)))

118755.0
